In [44]:
import nltk
import pandas as pd
import random
pd.set_option('display.max_colwidth',100)
import re
import string
import gensim
#nltk.download()
#python -m nltk.downloader all - from command line
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from collections import defaultdict
from nltk import pos_tag
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim import corpora,models
from gensim.models import LsiModel

In [ ]:
#Preparing dataset

In [13]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

In [ ]:
#Text Preprocessing

In [59]:
# tokenizer = RegexpTokenizer(r'\w+')
# stop_words = stopwords.words('english')
# lemmatizer = WordNetLemmatizer()
# texts = []
# for i in doc_set:
#     raw = i.lower()
#     tokens = tokenizer.tokenize(raw)
#     stopped_tokens = [i for i in tokens if not i in stop_words]
#     lemmatized_tokens = ' '.join([lemmatizer.lemmatize(i) for i in stopped_tokens])
#     texts.append(lemmatized_tokens)
# print(texts)

In [21]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
texts = []
for i in doc_set:
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(i) for i in stopped_tokens]
    texts.append(lemmatized_tokens)
print(texts)

[['brocolli', 'good', 'eat', 'brother', 'like', 'eat', 'good', 'brocolli', 'mother'], ['mother', 'spends', 'lot', 'time', 'driving', 'brother', 'around', 'baseball', 'practice'], ['health', 'expert', 'suggest', 'driving', 'may', 'cause', 'increased', 'tension', 'blood', 'pressure'], ['often', 'feel', 'pressure', 'perform', 'well', 'school', 'mother', 'never', 'seems', 'drive', 'brother', 'better'], ['health', 'professional', 'say', 'brocolli', 'good', 'health']]


In [ ]:
#doc2bow

In [57]:
# #Vectorization
# Count_vect = CountVectorizer()
# Count_vect.fit(texts)
# Doc_term_matrix_count_vectorizer = Count_vect.transform(texts)
# print('Representation of Corpus:')
# print(pd.DataFrame(Doc_term_matrix_count_vectorizer.toarray(),columns=Count_vect.get_feature_names()))

# #Model Building
# svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
# svd_model.fit(Doc_term_matrix_count_vectorizer)
# terms = Count_vect.get_feature_names()
# for i, comp in enumerate(svd_model.components_):
#     terms_comp = zip(terms, comp)
#     sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:4]
#     print("Topic "+str(i)+": ")
#     for t in sorted_terms:
#         print(t[0],t[1])

In [45]:
#Vectorization
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]
print(dictionary)
print(bow_corpus)

#Model Building
lsamodel_bow_corpus = LsiModel(bow_corpus, num_topics=2,id2word = dictionary)
# corpus ({iterable of list of (int, float), scipy.sparse.csc}, optional) – Stream of document vectors or 
# a sparse matrix of shape (num_documents, num_terms).
# num_topics (int, optional) – Number of requested factors (latent dimensions)
# id2word (dict of {int: str}, optional) – ID to word mapping, optional.
# chunksize (int, optional) – Number of documents to be used in each training chunk.
# decay (float, optional) – Weight of existing observations relatively to new ones.
# distributed (bool, optional) – If True - distributed mode (parallel execution on several machines) will be used.
# onepass (bool, optional) – Whether the one-pass algorithm should be used for training. 
# Pass False to force a multi-pass stochastic algorithm.
# power_iters (int, optional) – Number of power iteration steps to be used. 
# Increasing the number of power iterations improves accuracy, but lowers performance
# extra_samples (int, optional) – Extra samples to be used besides the rank k. Can improve accuracy.
# dtype (type, optional) – Enforces a type for elements of the decomposed matrix
print(lsamodel_bow_corpus.print_topics(num_topics=2, num_words=3))

#Evaluating the model
coherence_model_lda = models.CoherenceModel(model=lsamodel_bow_corpus, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = models.CoherenceModel(model=lsamodel_bow_corpus, texts=texts, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Dictionary(33 unique tokens: ['brocolli', 'brother', 'eat', 'good', 'like']...)
[[(0, 2), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1)], [(1, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(8, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(1, 1), (5, 1), (19, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(0, 1), (3, 1), (16, 2), (31, 1), (32, 1)]]
[(0, '0.457*"good" + 0.457*"brocolli" + 0.377*"eat"'), (1, '-0.305*"good" + -0.305*"brocolli" + 0.299*"pressure"')]

Coherence Score:  0.3160210901215216

Coherence Score:  -12.776541787339937


In [ ]:
#Tf idf

In [58]:
# # Vectorization
# from sklearn.feature_extraction.text import TfidfVectorizer
# Tfidf_vect = TfidfVectorizer()
# Tfidf_vect.fit(texts)
# Doc_term_matrix_tfidf_vectorizer = Tfidf_vect.transform(texts)
# print('Representation of Corpus:')
# print(pd.DataFrame(Doc_term_matrix_tfidf_vectorizer.toarray(),columns=Tfidf_vect.get_feature_names()))

# # Model Building
# svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
# svd_model.fit(Doc_term_matrix_tfidf_vectorizer)
# terms = Tfidf_vect.get_feature_names()
# for i, comp in enumerate(svd_model.components_):
#     terms_comp = zip(terms, comp)
#     sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:4]
#     print("Topic "+str(i)+": ")
#     for t in sorted_terms:
#         print(t[0],t[1])
        

In [47]:
#Vectorization
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]
print(dictionary)
for doc in tfidf_corpus:
    print(doc)

#Model Building
lsamodel_tfidf_corpus = LsiModel(tfidf_corpus, num_topics=2,id2word = dictionary)
print(lsamodel_tfidf_corpus.print_topics(num_topics=2, num_words=3))

#Evaluating the model
coherence_model_lda = models.CoherenceModel(model=lsamodel_tfidf_corpus, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = models.CoherenceModel(model=lsamodel_tfidf_corpus, texts=texts, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Dictionary(33 unique tokens: ['brocolli', 'brother', 'eat', 'good', 'like']...)
[(0, 0.40784451109112935), (1, 0.11368521994734913), (2, 0.7163669735975946), (3, 0.40784451109112935), (4, 0.3581834867987973), (5, 0.11368521994734913)]
[(1, 0.12424759593709131), (5, 0.12424759593709131), (6, 0.3914619434234833), (7, 0.3914619434234833), (8, 0.2228684610131362), (9, 0.3914619434234833), (10, 0.3914619434234833), (11, 0.3914619434234833), (12, 0.3914619434234833)]
[(8, 0.2016345105176491), (13, 0.35416512946544426), (14, 0.35416512946544426), (15, 0.35416512946544426), (16, 0.2016345105176491), (17, 0.35416512946544426), (18, 0.35416512946544426), (19, 0.2016345105176491), (20, 0.35416512946544426), (21, 0.35416512946544426)]
[(1, 0.10283764444679584), (5, 0.10283764444679584), (19, 0.18446447498008656), (22, 0.32400646345397865), (23, 0.32400646345397865), (24, 0.32400646345397865), (25, 0.32400646345397865), (26, 0.32400646345397865), (27, 0.32400646345397865), (28, 0.32400646345397865)